In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dc5a77c4",
   "metadata": {},
   "outputs": [],
   "source": [
    "!pip install numpy==1.23.1\n",
    "!pip install pandas==1.4.3\n",
    "!pip install matplotlib==3.3.2\n",
    "!pip install seaborn==0.11.0\n",
    "!pip install joblib==1.1.0\n",
    "!pip install nltk==3.7\n",
    "!pip install wordcloud==1.8.2.2\n",
    "!pip install scikit_learn==1.0.2\n",
    "!pip install scipy==1.9.0"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Contents:"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "I. [Loading the Data:](#Loading-the-Data:)\n",
    "\n",
    "II. [Helper functions to prepare the data:](#Helper-functions-to-prepare-the-data:)\n",
    "\n",
    "III. [Test Inference:](#Test-Inference:)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Loading the Data:"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "([Contents:](#Contents:))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "pd.set_option('display.max_rows',None)\n",
    "pd.set_option('display.max_columns',None)\n",
    "\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "import os\n",
    "import time \n",
    "import re\n",
    "\n",
    "import sklearn\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.linear_model import LinearRegression, Lasso, Ridge\n",
    "from sklearn import metrics\n",
    "\n",
    "from scipy import stats\n",
    "\n",
    "import requests\n",
    "import pickle\n",
    "import joblib\n",
    "\n",
    "import re\n",
    "from wordcloud import WordCloud\n",
    "import nltk\n",
    "from nltk.corpus import stopwords\n",
    "from sklearn.feature_extraction.text import CountVectorizer\n",
    "from collections import Counter\n",
    "\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Helper functions to prepare the data:"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "([Contents:](#Contents:))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# average propery area (feature cleaning)\n",
    "def avg_property_area(x):\n",
    "    numbers = re.compile(r\"[-+]?(\\d*\\.\\d+|\\d+)\") \n",
    "    x = numbers.findall(x)\n",
    "    if len(x) == 1:\n",
    "        return np.float(x[0])\n",
    "    elif len(x) == 2:\n",
    "        return (np.float(x[0])+np.float(x[1]))/2\n",
    "    else:\n",
    "        return -99\n",
    "    \n",
    "# Outlier treatment\n",
    "def clip_outliers(df,col):\n",
    "    q_l = df[col].quantile(0.25)\n",
    "    q_h = df[col].quantile(0.95)\n",
    "    df[col] = df[col].clip(lower = q_l, upper = q_h)\n",
    "    return df    \n",
    "\n",
    "# Text cleaning\n",
    "# Preprocessing the text data\n",
    "REPLACE_BY_SPACE_RE = re.compile(\"[/(){}\\[\\]\\|@,;!]\")\n",
    "BAD_SYMBOLS_RE = re.compile(\"[^0-9a-z #+_]\")\n",
    "STOPWORDS_nlp = set(stopwords.words('english'))\n",
    "\n",
    "#Custom Stoplist\n",
    "stoplist = [\"i\",\"project\",\"living\",\"home\",'apartment',\"pune\",\"me\",\"my\",\"myself\",\"we\",\"our\",\"ours\",\"ourselves\",\"you\",\"you're\",\"you've\",\"you'll\",\"you'd\",\"your\",\n",
    "            \"yours\",\"yourself\",\"yourselves\",\"he\",\"him\",\"his\",\"himself\",\"she\",\"she's\",\"her\",\"hers\",\"herself\",\"it\",\n",
    "            \"it's\",\"its\",\"itself\",\"they\",\"them\",\"their\",\"theirs\",\"themselves\",\"what\",\"which\",\"who\",\"whom\",\"this\",\"that\",\"that'll\",\n",
    "            \"these\",\"those\",\"am\",\"is\",\"are\",\"was\",\"were\",\"be\",\"been\",\"being\",\"have\",\"has\",\"had\",\"having\",\"do\",\"does\",\"did\",\n",
    "            \"doing\",\"a\",\"an\",\"the\",\"and\",\"but\",\"if\",\"or\",\"because\",\"as\",\"until\",\"while\",\"of\",\"at\",\"by\",\"for\",\"with\",\"about\",\n",
    "            \"against\",\"between\",\"into\",\"through\",\"during\",\"before\",\"after\",\"above\",\"below\",\"to\",\"from\",\"up\",\"down\",\"in\",\"out\",\n",
    "            \"on\",\"off\",\"over\",\"under\",\"again\",\"further\",\"then\",\"once\",\"here\",\"there\",\"when\",\"where\",\"why\",\"all\",\"any\",\n",
    "            \"both\",\"each\",\"few\",\"more\",\"most\",\"other\",\"some\",\"such\",\"no\",\"nor\",\"not\",\"only\",\"own\",\"same\",\"so\",\"than\",\"too\",\n",
    "            \"very\",\"s\",\"t\",\"can\",\"will\",\"just\",\"don\",\"don't\",\"should\",\"should've\",\"now\",\"d\",\"ll\",\"m\",\"o\",\"re\",\"ve\",\"y\",\"ain\",\n",
    "            \"aren\",\"couldn\",\"didn\",\"doesn\",\"hadn\",\"hasn\",\n",
    "            \"haven\",\"isn\",\"ma\",\"mightn\",\"mustn\",\"needn\",\"shan\",\"shan't\",\n",
    "            \"shouldn\",\"wasn\",\"weren\",\"won\",\"rt\",\"rt\",\"qt\",\"for\",\n",
    "            \"the\",\"with\",\"in\",\"of\",\"and\",\"its\",\"it\",\"this\",\"i\",\"have\",\"has\",\"would\",\"could\",\"you\",\"a\",\"an\",\n",
    "            \"be\",\"am\",\"can\",\"edushopper\",\"will\",\"to\",\"on\",\"is\",\"by\",\"ive\",\"im\",\"your\",\"we\",\"are\",\"at\",\"as\",\"any\",\"ebay\",\"thank\",\"hello\",\"know\",\n",
    "            \"need\",\"want\",\"look\",\"hi\",\"sorry\",\"http\", \"https\",\"body\",\"dear\",\"hello\",\"hi\",\"thanks\",\"sir\",\"tomorrow\",\"sent\",\"send\",\"see\",\"there\",\"welcome\",\"what\",\"well\",\"us\"]\n",
    "\n",
    "STOPWORDS_nlp.update(stoplist)\n",
    "\n",
    "# Function to preprocess the text\n",
    "def text_prepare(text):\n",
    "    \"\"\"\n",
    "        text: a string\n",
    "        \n",
    "        return: modified initial string\n",
    "    \"\"\"\n",
    "    text = text.replace(\"\\d+\",\" \") # removing digits\n",
    "    text = re.sub(r\"(?:\\@|https?\\://)\\S+\", \"\", text) #removing mentions and urls\n",
    "    text = text.lower() # lowercase text\n",
    "    text =  re.sub('[0-9]+', '', text)\n",
    "    text = REPLACE_BY_SPACE_RE.sub(\" \", text) # replace REPLACE_BY_SPACE_RE symbols by space in text\n",
    "    text = BAD_SYMBOLS_RE.sub(\" \", text) # delete symbols which are in BAD_SYMBOLS_RE from text\n",
    "    text = ' '.join([word for word in text.split() if word not in STOPWORDS_nlp]) # delete stopwors from text\n",
    "    text = text.strip()\n",
    "    return text\n",
    "\n",
    "# Pos counter\n",
    "def pos_counter(x,pos):\n",
    "    \"\"\"\n",
    "    Returns the count for the given parts of speech tag\n",
    "    \n",
    "    NN - Noun\n",
    "    VB - Verb\n",
    "    JJ - Adjective\n",
    "    RB - Adverb\n",
    "    \"\"\"\n",
    "    tokens = nltk.word_tokenize(x.lower())\n",
    "    tokens = [word for word in tokens if word not in STOPWORDS_nlp]\n",
    "    text = nltk.Text(tokens)\n",
    "    tags = nltk.pos_tag(text)\n",
    "    counts = Counter(tag for word,tag in tags)\n",
    "    return counts[pos]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "def preprocess(df):\n",
    "    # Extracting State and Country separately from the Location Column\n",
    "    df['City'] = df['Location'].apply(lambda x: x.split(',')[0].lower().strip())\n",
    "    df['State'] = df['Location'].apply(lambda x: x.split(',')[1].lower().strip())\n",
    "    df['Country'] = df['Location'].apply(lambda x: x.split(',')[2].lower().strip())\n",
    "    \n",
    "    # Regex to match the numbers and create a separate column\n",
    "    numbers = re.compile(r\"[-+]?(\\d*\\.\\d+|\\d+)\") \n",
    "    df['Property Type Cleaned'] = df['Propert Type'].apply(lambda x: numbers.findall(x)[0] \n",
    "                                                           if len(numbers.findall(x)) > 0 else 0)\n",
    "    \n",
    "    # Cleaning the text columns\n",
    "    df['Sub-Area Cleaned'] = df['Sub-Area'].apply(lambda x: x.lower().strip())\n",
    "    df['Company Name Cleaned'] = df['Company Name'].apply(lambda x: x.lower().strip())\n",
    "    df['TownShip Name/ Society Name Cleaned'] = df['TownShip Name/ Society Name'].apply(lambda x: x.lower().strip())\n",
    "    df['Description Cleaned'] = df['Description'].apply(lambda x: x.lower().strip())\n",
    "    \n",
    "    \n",
    "    # Cleaning and encoding Binary Features\n",
    "    df['ClubHouse Cleaned'] = (df['ClubHouse'].apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))\n",
    "    df['School / University in Township Cleaned'] = (df['School / University in Township ']\n",
    "                                                         .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))\n",
    "    df['Hospital in TownShip Cleaned'] = (df['Hospital in TownShip']\n",
    "                                                         .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))\n",
    "    df['Mall in TownShip Cleaned'] = (df['Mall in TownShip']\n",
    "                                                         .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))\n",
    "    df['Park / Jogging track Cleaned'] = (df['Park / Jogging track']\n",
    "                                                         .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))\n",
    "    df['Swimming Pool Cleaned'] = (df['Swimming Pool']\n",
    "                                                     .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))\n",
    "    df['Gym Cleaned'] = (df['Gym']\n",
    "                                 .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0})) \n",
    "    \n",
    "    \n",
    "\n",
    "    # Cleaning numerical columns\n",
    "    numbers = re.compile(r\"[-+]?(\\d*\\.\\d+|\\d+)\")     \n",
    "    df['Property Area in Sq. Ft. Cleaned'] = df['Property Area in Sq. Ft.'].apply(lambda x: avg_property_area(str(x)))\n",
    "#     df['Price in lakhs Cleaned'] = (df['Price in lakhs'].apply(lambda x: np.float(numbers.findall(str(x))[0]) \n",
    "#                                                                if len(numbers.findall(str(x)))>0 else np.nan ))\n",
    "    \n",
    "    # Selecting the requried columns\n",
    "    features = df.columns.tolist()[18:]\n",
    "    df1 = df[features]\n",
    "    df_final = df1.dropna()\n",
    "    \n",
    "    return df_final"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_features(df):\n",
    "    # outlier treatment\n",
    "    # Treating outliers in the numeric columns\n",
    "    cols_to_treat = ['Property Area in Sq. Ft. Cleaned']\n",
    "    \n",
    "    for col in cols_to_treat:\n",
    "        df = clip_outliers(df,col)\n",
    "    \n",
    "    # creating the price by sub-area feature\n",
    "    fileName = 'model/sub_area_price_map.pkl'\n",
    "    with open(fileName,'rb') as f:\n",
    "        sub_area_price_map = pickle.load(f)    \n",
    "    df['Price by sub-area'] =  df['Sub-Area Cleaned'].map(sub_area_price_map)\n",
    "    \n",
    "    # Adding the amenitites score feature\n",
    "    amenities_col = ['ClubHouse Cleaned',\n",
    "                     'School / University in Township Cleaned',\n",
    "                     'Hospital in TownShip Cleaned',\n",
    "                     'Mall in TownShip Cleaned',\n",
    "                     'Park / Jogging track Cleaned',\n",
    "                     'Swimming Pool Cleaned',\n",
    "                     'Gym Cleaned']\n",
    "    temp = df[amenities_col]\n",
    "    temp['Amenities score'] = temp.sum(axis=1)\n",
    "    df['Amenities score'] = temp['Amenities score']\n",
    "    \n",
    "    # creating the price by amenities score feature\n",
    "    fileName = 'model/amenities_score_price_map.pkl'\n",
    "    with open(fileName,'rb') as f:\n",
    "        amenities_score_price_map = pickle.load(f)  \n",
    "        \n",
    "    df['Price by Amenities score'] =  df['Amenities score'].map(amenities_score_price_map)\n",
    "    \n",
    "    # cleaning the description column and creating pos features\n",
    "    df[\"Description Cleaned\"] =  df[\"Description Cleaned\"].astype(str).apply(text_prepare)\n",
    "    df['Noun_Counts'] = df['Description Cleaned'].apply(lambda x: pos_counter(x,'NN'))\n",
    "    df['Verb_Counts'] = df['Description Cleaned'].apply(lambda x: (pos_counter(x,'VB')+pos_counter(x,'RB')))\n",
    "    df['Adjective_Counts'] = df['Description Cleaned'].apply(lambda x: pos_counter(x,'JJ'))\n",
    "    \n",
    "    # Ngram features\n",
    "    fileName = 'model/count_vectorizer.pkl'\n",
    "    with open(fileName,'rb') as f:\n",
    "        cv_object = pickle.load(f)\n",
    "    \n",
    "    X = cv_object.transform(df['Description Cleaned'])\n",
    "    df_ngram = pd.DataFrame(X.toarray(),columns=cv_object.get_feature_names())\n",
    "     \n",
    "    # Adding this to the main dataframe\n",
    "    df_final = pd.concat([df.reset_index(drop=True),df_ngram.reset_index(drop=True)],axis=1)\n",
    "    \n",
    "    # selecting the final model ready features\n",
    "    fileName = 'model/raw_features_mapping.pkl'\n",
    "    with open(fileName,'rb') as f:\n",
    "        feature_mapping = pickle.load(f)   \n",
    "        \n",
    "    fileName = 'model/features.pkl'\n",
    "    with open(fileName,'rb') as f:\n",
    "        feature_list = pickle.load(f)           \n",
    "    \n",
    "    # Removing price column as it is not available in test data\n",
    "    feature_list.remove('Price_in_lakhs')\n",
    "\n",
    "    df_final = df_final.rename(columns=feature_mapping)\n",
    "    df_final = df_final[feature_list]\n",
    "    \n",
    "    return df_final"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Test Inference:"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "([Contents:](#Contents:))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(200, 16)\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Sr. No.</th>\n",
       "      <th>Location</th>\n",
       "      <th>Sub-Area</th>\n",
       "      <th>Propert Type</th>\n",
       "      <th>Property Area in Sq. Ft.</th>\n",
       "      <th>Company Name</th>\n",
       "      <th>TownShip Name/ Society Name</th>\n",
       "      <th>Total TownShip Area in Acres</th>\n",
       "      <th>ClubHouse</th>\n",
       "      <th>School / University in Township</th>\n",
       "      <th>Hospital in TownShip</th>\n",
       "      <th>Mall in TownShip</th>\n",
       "      <th>Park / Jogging track</th>\n",
       "      <th>Swimming Pool</th>\n",
       "      <th>Gym</th>\n",
       "      <th>Description</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1</td>\n",
       "      <td>Pune, Maharashtra, India</td>\n",
       "      <td>Bavdhan</td>\n",
       "      <td>1 BHK</td>\n",
       "      <td>492</td>\n",
       "      <td>Shapoorji Paloonji</td>\n",
       "      <td>Vanaha</td>\n",
       "      <td>1000.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Shapoorji Paloonji comunity located in the sub...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2</td>\n",
       "      <td>Pune, Maharashtra, India</td>\n",
       "      <td>Bavdhan</td>\n",
       "      <td>2 BHK</td>\n",
       "      <td>774</td>\n",
       "      <td>Shapoorji Paloonji</td>\n",
       "      <td>Vanaha</td>\n",
       "      <td>1000.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Vanaha Township located near the lonavala hill...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3</td>\n",
       "      <td>Pune, Maharashtra, India</td>\n",
       "      <td>Bavdhan</td>\n",
       "      <td>3 BHK</td>\n",
       "      <td>889</td>\n",
       "      <td>Shapoorji Paloonji</td>\n",
       "      <td>Vanaha</td>\n",
       "      <td>1000.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Vanaha Society is suitable for all aged group ...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>4</td>\n",
       "      <td>Pune, Maharashtra, India</td>\n",
       "      <td>Bavdhan</td>\n",
       "      <td>3 BHK Grand</td>\n",
       "      <td>1018</td>\n",
       "      <td>Shapoorji Paloonji</td>\n",
       "      <td>Vanaha</td>\n",
       "      <td>1000.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Vanaha township are offering 3BHK grand prpoer...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>5</td>\n",
       "      <td>Pune, Maharashtra, India</td>\n",
       "      <td>Mahalunge</td>\n",
       "      <td>2BHK</td>\n",
       "      <td>743</td>\n",
       "      <td>Godrej Properties</td>\n",
       "      <td>Godrej Hills retreat</td>\n",
       "      <td>100.0</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>Yes</td>\n",
       "      <td>The area is a hub of prestigious schools like ...</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   Sr. No.                  Location   Sub-Area Propert Type  \\\n",
       "0        1  Pune, Maharashtra, India    Bavdhan        1 BHK   \n",
       "1        2  Pune, Maharashtra, India    Bavdhan        2 BHK   \n",
       "2        3  Pune, Maharashtra, India    Bavdhan        3 BHK   \n",
       "3        4  Pune, Maharashtra, India    Bavdhan  3 BHK Grand   \n",
       "4        5  Pune, Maharashtra, India  Mahalunge         2BHK   \n",
       "\n",
       "  Property Area in Sq. Ft.        Company Name TownShip Name/ Society Name  \\\n",
       "0                      492  Shapoorji Paloonji                     Vanaha    \n",
       "1                      774  Shapoorji Paloonji                     Vanaha    \n",
       "2                      889  Shapoorji Paloonji                     Vanaha    \n",
       "3                     1018  Shapoorji Paloonji                     Vanaha    \n",
       "4                      743   Godrej Properties        Godrej Hills retreat   \n",
       "\n",
       "   Total TownShip Area in Acres ClubHouse School / University in Township   \\\n",
       "0                        1000.0       Yes                              Yes   \n",
       "1                        1000.0       Yes                              Yes   \n",
       "2                        1000.0       Yes                              Yes   \n",
       "3                        1000.0       Yes                              Yes   \n",
       "4                         100.0       Yes                              Yes   \n",
       "\n",
       "  Hospital in TownShip Mall in TownShip Park / Jogging track Swimming Pool  \\\n",
       "0                  Yes              Yes                  Yes           Yes   \n",
       "1                  Yes              Yes                  Yes           Yes   \n",
       "2                  Yes              Yes                  Yes           Yes   \n",
       "3                  Yes              Yes                  Yes           Yes   \n",
       "4                  Yes              Yes                  Yes           Yes   \n",
       "\n",
       "   Gym                                        Description  \n",
       "0  Yes  Shapoorji Paloonji comunity located in the sub...  \n",
       "1  Yes  Vanaha Township located near the lonavala hill...  \n",
       "2  Yes  Vanaha Society is suitable for all aged group ...  \n",
       "3  Yes  Vanaha township are offering 3BHK grand prpoer...  \n",
       "4  Yes  The area is a hub of prestigious schools like ...  "
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Loading the data\n",
    "data= pd.read_excel('data/Pune Real Estate Data.xlsx')\n",
    "data = data.drop(['Price in Millions','Price in lakhs'],axis=1)\n",
    "print(data.shape)\n",
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "df_preprocess = preprocess(data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(200, 14)\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Country</th>\n",
       "      <th>Property Type Cleaned</th>\n",
       "      <th>Sub-Area Cleaned</th>\n",
       "      <th>Company Name Cleaned</th>\n",
       "      <th>TownShip Name/ Society Name Cleaned</th>\n",
       "      <th>Description Cleaned</th>\n",
       "      <th>ClubHouse Cleaned</th>\n",
       "      <th>School / University in Township Cleaned</th>\n",
       "      <th>Hospital in TownShip Cleaned</th>\n",
       "      <th>Mall in TownShip Cleaned</th>\n",
       "      <th>Park / Jogging track Cleaned</th>\n",
       "      <th>Swimming Pool Cleaned</th>\n",
       "      <th>Gym Cleaned</th>\n",
       "      <th>Property Area in Sq. Ft. Cleaned</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>india</td>\n",
       "      <td>1</td>\n",
       "      <td>bavdhan</td>\n",
       "      <td>shapoorji paloonji</td>\n",
       "      <td>vanaha</td>\n",
       "      <td>shapoorji paloonji comunity located in the sub...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>492.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>india</td>\n",
       "      <td>2</td>\n",
       "      <td>bavdhan</td>\n",
       "      <td>shapoorji paloonji</td>\n",
       "      <td>vanaha</td>\n",
       "      <td>vanaha township located near the lonavala hill...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>774.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>india</td>\n",
       "      <td>3</td>\n",
       "      <td>bavdhan</td>\n",
       "      <td>shapoorji paloonji</td>\n",
       "      <td>vanaha</td>\n",
       "      <td>vanaha society is suitable for all aged group ...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>889.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>india</td>\n",
       "      <td>3</td>\n",
       "      <td>bavdhan</td>\n",
       "      <td>shapoorji paloonji</td>\n",
       "      <td>vanaha</td>\n",
       "      <td>vanaha township are offering 3bhk grand prpoer...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1018.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>india</td>\n",
       "      <td>2</td>\n",
       "      <td>mahalunge</td>\n",
       "      <td>godrej properties</td>\n",
       "      <td>godrej hills retreat</td>\n",
       "      <td>the area is a hub of prestigious schools like ...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>743.0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  Country Property Type Cleaned Sub-Area Cleaned Company Name Cleaned  \\\n",
       "0   india                     1          bavdhan   shapoorji paloonji   \n",
       "1   india                     2          bavdhan   shapoorji paloonji   \n",
       "2   india                     3          bavdhan   shapoorji paloonji   \n",
       "3   india                     3          bavdhan   shapoorji paloonji   \n",
       "4   india                     2        mahalunge    godrej properties   \n",
       "\n",
       "  TownShip Name/ Society Name Cleaned  \\\n",
       "0                              vanaha   \n",
       "1                              vanaha   \n",
       "2                              vanaha   \n",
       "3                              vanaha   \n",
       "4                godrej hills retreat   \n",
       "\n",
       "                                 Description Cleaned  ClubHouse Cleaned  \\\n",
       "0  shapoorji paloonji comunity located in the sub...                  1   \n",
       "1  vanaha township located near the lonavala hill...                  1   \n",
       "2  vanaha society is suitable for all aged group ...                  1   \n",
       "3  vanaha township are offering 3bhk grand prpoer...                  1   \n",
       "4  the area is a hub of prestigious schools like ...                  1   \n",
       "\n",
       "   School / University in Township Cleaned  Hospital in TownShip Cleaned  \\\n",
       "0                                        1                             1   \n",
       "1                                        1                             1   \n",
       "2                                        1                             1   \n",
       "3                                        1                             1   \n",
       "4                                        1                             1   \n",
       "\n",
       "   Mall in TownShip Cleaned  Park / Jogging track Cleaned  \\\n",
       "0                         1                             1   \n",
       "1                         1                             1   \n",
       "2                         1                             1   \n",
       "3                         1                             1   \n",
       "4                         1                             1   \n",
       "\n",
       "   Swimming Pool Cleaned  Gym Cleaned  Property Area in Sq. Ft. Cleaned  \n",
       "0                      1            1                             492.0  \n",
       "1                      1            1                             774.0  \n",
       "2                      1            1                             889.0  \n",
       "3                      1            1                            1018.0  \n",
       "4                      1            1                             743.0  "
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "print(df_preprocess.shape)\n",
    "df_preprocess.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "df_features = create_features(df_preprocess)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(200, 25)\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>PropertyType</th>\n",
       "      <th>ClubHouse</th>\n",
       "      <th>School_University_in_Township</th>\n",
       "      <th>Hospital_in_TownShip</th>\n",
       "      <th>Mall_in_TownShip</th>\n",
       "      <th>Park_Jogging_track</th>\n",
       "      <th>Swimming_Pool</th>\n",
       "      <th>Gym</th>\n",
       "      <th>Property_Area_in_Sq_Ft</th>\n",
       "      <th>Price_by_sub_area</th>\n",
       "      <th>Amenities_score</th>\n",
       "      <th>Price_by_Amenities_score</th>\n",
       "      <th>Noun_Counts</th>\n",
       "      <th>Verb_Counts</th>\n",
       "      <th>Adjective_Counts</th>\n",
       "      <th>boasts_elegant</th>\n",
       "      <th>elegant_towers</th>\n",
       "      <th>every_day</th>\n",
       "      <th>great_community</th>\n",
       "      <th>mantra_gold</th>\n",
       "      <th>offering_bedroom</th>\n",
       "      <th>quality_specification</th>\n",
       "      <th>stories_offering</th>\n",
       "      <th>towers_stories</th>\n",
       "      <th>world_class</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>670.0</td>\n",
       "      <td>58.044000</td>\n",
       "      <td>7</td>\n",
       "      <td>72.666667</td>\n",
       "      <td>9</td>\n",
       "      <td>1</td>\n",
       "      <td>3</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>774.0</td>\n",
       "      <td>58.044000</td>\n",
       "      <td>7</td>\n",
       "      <td>72.666667</td>\n",
       "      <td>9</td>\n",
       "      <td>1</td>\n",
       "      <td>3</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>3</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>889.0</td>\n",
       "      <td>58.044000</td>\n",
       "      <td>7</td>\n",
       "      <td>72.666667</td>\n",
       "      <td>9</td>\n",
       "      <td>1</td>\n",
       "      <td>3</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1018.0</td>\n",
       "      <td>58.044000</td>\n",
       "      <td>7</td>\n",
       "      <td>72.666667</td>\n",
       "      <td>8</td>\n",
       "      <td>1</td>\n",
       "      <td>3</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>2</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>743.0</td>\n",
       "      <td>73.555556</td>\n",
       "      <td>7</td>\n",
       "      <td>72.666667</td>\n",
       "      <td>12</td>\n",
       "      <td>1</td>\n",
       "      <td>6</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  PropertyType  ClubHouse  School_University_in_Township  \\\n",
       "0            1          1                              1   \n",
       "1            2          1                              1   \n",
       "2            3          1                              1   \n",
       "3            3          1                              1   \n",
       "4            2          1                              1   \n",
       "\n",
       "   Hospital_in_TownShip  Mall_in_TownShip  Park_Jogging_track  Swimming_Pool  \\\n",
       "0                     1                 1                   1              1   \n",
       "1                     1                 1                   1              1   \n",
       "2                     1                 1                   1              1   \n",
       "3                     1                 1                   1              1   \n",
       "4                     1                 1                   1              1   \n",
       "\n",
       "   Gym  Property_Area_in_Sq_Ft  Price_by_sub_area  Amenities_score  \\\n",
       "0    1                   670.0          58.044000                7   \n",
       "1    1                   774.0          58.044000                7   \n",
       "2    1                   889.0          58.044000                7   \n",
       "3    1                  1018.0          58.044000                7   \n",
       "4    1                   743.0          73.555556                7   \n",
       "\n",
       "   Price_by_Amenities_score  Noun_Counts  Verb_Counts  Adjective_Counts  \\\n",
       "0                 72.666667            9            1                 3   \n",
       "1                 72.666667            9            1                 3   \n",
       "2                 72.666667            9            1                 3   \n",
       "3                 72.666667            8            1                 3   \n",
       "4                 72.666667           12            1                 6   \n",
       "\n",
       "   boasts_elegant  elegant_towers  every_day  great_community  mantra_gold  \\\n",
       "0               0               0          0                0            0   \n",
       "1               0               0          0                0            0   \n",
       "2               0               0          0                0            0   \n",
       "3               0               0          0                0            0   \n",
       "4               0               0          0                0            0   \n",
       "\n",
       "   offering_bedroom  quality_specification  stories_offering  towers_stories  \\\n",
       "0                 0                      0                 0               0   \n",
       "1                 0                      0                 0               0   \n",
       "2                 0                      0                 0               0   \n",
       "3                 0                      0                 0               0   \n",
       "4                 0                      0                 0               0   \n",
       "\n",
       "   world_class  \n",
       "0            0  \n",
       "1            0  \n",
       "2            0  \n",
       "3            0  \n",
       "4            0  "
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "print(df_features.shape)\n",
    "df_features.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['PropertyType',\n",
       " 'ClubHouse',\n",
       " 'School_University_in_Township',\n",
       " 'Hospital_in_TownShip',\n",
       " 'Mall_in_TownShip']"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "columns = df_features.columns.tolist()\n",
    "columns[:5]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "payload = df_features.iloc[3].to_dict()\n",
    "#Even if an integer of the type int64 is present in another object like a dictionary, \n",
    "#the TypeError exception will occur with the message “TypeError: Object of type int64 is not JSON serializable”\n",
    "import json\n",
    "# define a class to avoid that\n",
    "class NpEncoder(json.JSONEncoder):\n",
    "    def default(self, obj):\n",
    "        if isinstance(obj, np.integer):\n",
    "            return int(obj)\n",
    "        if isinstance(obj, np.floating):\n",
    "            return float(obj)\n",
    "        if isinstance(obj, np.ndarray):\n",
    "            return obj.tolist()\n",
    "        return super(NpEncoder, self).default(obj)\n",
    "    \n",
    "payload = json.dumps(payload,cls=NpEncoder)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'{\"PropertyType\": \"3\", \"ClubHouse\": 1, \"School_University_in_Township\": 1, \"Hospital_in_TownShip\": 1, \"Mall_in_TownShip\": 1, \"Park_Jogging_track\": 1, \"Swimming_Pool\": 1, \"Gym\": 1, \"Property_Area_in_Sq_Ft\": 1018.0, \"Price_by_sub_area\": 58.044000000000004, \"Amenities_score\": 7, \"Price_by_Amenities_score\": 72.66666666666667, \"Noun_Counts\": 8, \"Verb_Counts\": 1, \"Adjective_Counts\": 3, \"boasts_elegant\": 0, \"elegant_towers\": 0, \"every_day\": 0, \"great_community\": 0, \"mantra_gold\": 0, \"offering_bedroom\": 0, \"quality_specification\": 0, \"stories_offering\": 0, \"towers_stories\": 0, \"world_class\": 0}'"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "payload"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [],
   "source": [
    "payload = df_features.iloc[3].to_dict()\n",
    "payload = json.dumps(payload,cls=NpEncoder)\n",
    "\n",
    "out =  requests.post(url='https://property-price-prediction-live.herokuapp.com/predict',\n",
    "                data=payload)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'\"[85.55245708]\"'"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "out.text"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [],
   "source": [
    "output = []\n",
    "for i in range(len(df_features)):\n",
    "    payload = df_features.iloc[i].to_dict()\n",
    "    payload = json.dumps(payload,cls=NpEncoder)\n",
    "    \n",
    "    out =  requests.post(url='https://property-price-prediction-live.herokuapp.com/predict',\n",
    "                    data=payload)\n",
    "    result = np.float(re.sub('[^A-Za-z0-9.]+', '', out.text))  \n",
    "    output.append(result)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[41.73696173,\n",
       " 57.70878214,\n",
       " 74.58849812,\n",
       " 85.55245708,\n",
       " 66.79123803,\n",
       " 84.59893199,\n",
       " 77.66213533,\n",
       " 108.16028495,\n",
       " 78.39072133,\n",
       " 109.57364513,\n",
       " 159.10621088,\n",
       " 176.79921586,\n",
       " 173.62768353,\n",
       " 82.30834029,\n",
       " 118.12537567,\n",
       " 50.24301879,\n",
       " 57.96254756,\n",
       " 79.05972822,\n",
       " 52.32957137,\n",
       " 69.86078406,\n",
       " 74.73422689,\n",
       " 103.96509677,\n",
       " 38.42625687,\n",
       " 62.09826553,\n",
       " 52.03298601,\n",
       " 56.40783384,\n",
       " 67.41405046,\n",
       " 88.43816135,\n",
       " 71.24488624,\n",
       " 73.88559514,\n",
       " 83.5413806,\n",
       " 84.88514865,\n",
       " 103.69906684,\n",
       " 102.73270284,\n",
       " 115.79059498,\n",
       " 61.73645075,\n",
       " 106.18832373,\n",
       " 85.2035976,\n",
       " 119.28481463,\n",
       " 42.50511413,\n",
       " 74.52375717,\n",
       " 143.78489959,\n",
       " 30.68352346,\n",
       " 37.17655204,\n",
       " 41.66563326,\n",
       " 84.43268267,\n",
       " 87.32860919,\n",
       " 115.21966742,\n",
       " 109.0564247,\n",
       " 123.23834368,\n",
       " 168.31854671,\n",
       " 42.84774364,\n",
       " 51.65022681,\n",
       " 55.37759299,\n",
       " 65.17603304,\n",
       " 60.00438345,\n",
       " 48.02535026,\n",
       " 58.46073342,\n",
       " 64.29291026,\n",
       " 56.84451424,\n",
       " 86.81352401,\n",
       " 42.71811613,\n",
       " 53.57938083,\n",
       " 71.38925869,\n",
       " 68.10531012,\n",
       " 89.62387405,\n",
       " 80.54255645,\n",
       " 198.02413568,\n",
       " 206.04585597,\n",
       " 71.86346632,\n",
       " 75.42780719,\n",
       " 68.29750563,\n",
       " 71.6028411,\n",
       " 101.65920588,\n",
       " 98.28978398,\n",
       " 49.31402349,\n",
       " 58.67832153,\n",
       " 65.44468952,\n",
       " 101.17783324,\n",
       " 70.94985793,\n",
       " 92.79167626,\n",
       " 100.56553118,\n",
       " 36.69869583,\n",
       " 41.03465445,\n",
       " 57.51660476,\n",
       " 129.13718503,\n",
       " 104.83352665,\n",
       " 56.81860231,\n",
       " 54.15596765,\n",
       " 54.0140884,\n",
       " 52.78332523,\n",
       " 54.99632164,\n",
       " 147.1047694,\n",
       " 151.06983648,\n",
       " 158.17612746,\n",
       " 171.80572794,\n",
       " 184.14667896,\n",
       " 185.42457951,\n",
       " 62.6327785,\n",
       " 80.45007225,\n",
       " 87.38493708,\n",
       " 88.82832672,\n",
       " 114.71905331,\n",
       " 118.52342566,\n",
       " 133.89414118,\n",
       " 127.78941741,\n",
       " 49.82525388,\n",
       " 53.14347139,\n",
       " 41.20612597,\n",
       " 49.22784625,\n",
       " 48.46613121,\n",
       " 46.9374395,\n",
       " 47.70178535,\n",
       " 66.77560635,\n",
       " 99.57873886,\n",
       " 99.76561911,\n",
       " 104.72261629,\n",
       " 132.78680943,\n",
       " 143.78560808,\n",
       " 68.63605051,\n",
       " 88.45795607,\n",
       " 78.32650261,\n",
       " 91.80755637,\n",
       " 73.25928293,\n",
       " 70.71176549,\n",
       " 75.16217959,\n",
       " 101.13640178,\n",
       " 125.79157006,\n",
       " 156.31923186,\n",
       " 71.43341516,\n",
       " 69.13488019,\n",
       " 167.61816787,\n",
       " 54.22598083,\n",
       " 56.35135397,\n",
       " 64.0215926,\n",
       " 95.02977235,\n",
       " 91.6406071,\n",
       " 103.22321155,\n",
       " 131.98194994,\n",
       " 127.53614369,\n",
       " 135.76576043,\n",
       " 55.67023468,\n",
       " 64.32300233,\n",
       " 67.35364023,\n",
       " 88.65141817,\n",
       " 114.40368235,\n",
       " 69.98262209,\n",
       " 72.63450426,\n",
       " 80.3732065,\n",
       " 61.07579212,\n",
       " 60.83163938,\n",
       " 50.25739822,\n",
       " 52.82766646,\n",
       " 78.02168578,\n",
       " 80.09280294,\n",
       " 41.21853188,\n",
       " 46.27259709,\n",
       " 62.4600622,\n",
       " 92.84558114,\n",
       " 98.57878141,\n",
       " 99.41249173,\n",
       " 46.16730005,\n",
       " 58.74478641,\n",
       " 44.42937561,\n",
       " 52.42470837,\n",
       " 77.68093174,\n",
       " 81.77121323,\n",
       " 187.80992054,\n",
       " 199.21588711,\n",
       " 58.85402194,\n",
       " 81.20990984,\n",
       " 69.98181477,\n",
       " 99.05021164,\n",
       " 63.77388586,\n",
       " 47.40959022,\n",
       " 43.85190372,\n",
       " 45.8836447,\n",
       " 44.48565898,\n",
       " 56.04043329,\n",
       " 64.54846529,\n",
       " 71.17383046,\n",
       " 70.0195556,\n",
       " 146.50860221,\n",
       " 180.87147588,\n",
       " 188.94860414,\n",
       " 42.68429272,\n",
       " 42.28738761,\n",
       " 44.07964764,\n",
       " 56.25191475,\n",
       " 52.36277989,\n",
       " 46.44201944,\n",
       " 28.66659645,\n",
       " 28.94623141,\n",
       " 95.36841668,\n",
       " 127.8689963,\n",
       " 148.92776178,\n",
       " 175.39909445,\n",
       " 93.18527057,\n",
       " 131.97620132,\n",
       " 100.7738478]"
      ]
     },
     "execution_count": 17,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "output"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "# data= pd.read_excel(r'../data/Pune Real Estate Data.xlsx')\n",
    "# data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_prediction_interval(interval_estimate, prediction):\n",
    "    '''\n",
    "    Get a prediction interval for a linear regression model.\n",
    "    \n",
    "    INPUTS: \n",
    "        - interval_estimate based on the final model's performance on the training data \n",
    "        - predicted values for the test data,\n",
    "        - Prediction interval threshold (default = .95) \n",
    "    OUTPUT: \n",
    "        - Prediction interval for single test prediction\n",
    "    '''\n",
    "    \n",
    "    #generate prediction interval lower and upper bound cs_24\n",
    "    lower, upper = prediction - interval_estimate, prediction + interval_estimate\n",
    "    return lower, upper"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "31.826834460432945"
      ]
     },
     "execution_count": 20,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import pickle\n",
    "fileName = 'model/interval_est.pkl'\n",
    "with open(fileName,'rb') as f:\n",
    "    interval = pickle.load(f)\n",
    "\n",
    "interval"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "# getting prediction intervals for the test data\n",
    "lower_vet = []\n",
    "upper_vet = []\n",
    "\n",
    "for out in output:\n",
    "    lower, upper =  get_prediction_interval(interval,out)\n",
    "    lower_vet.append(lower)\n",
    "    upper_vet.append(upper)    "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>lower</th>\n",
       "      <th>upper</th>\n",
       "      <th>mean</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>9.910127</td>\n",
       "      <td>73.563796</td>\n",
       "      <td>41.736962</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>25.881948</td>\n",
       "      <td>89.535617</td>\n",
       "      <td>57.708782</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>42.761664</td>\n",
       "      <td>106.415333</td>\n",
       "      <td>74.588498</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>53.725623</td>\n",
       "      <td>117.379292</td>\n",
       "      <td>85.552457</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>34.964404</td>\n",
       "      <td>98.618072</td>\n",
       "      <td>66.791238</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>52.772098</td>\n",
       "      <td>116.425766</td>\n",
       "      <td>84.598932</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>45.835301</td>\n",
       "      <td>109.488970</td>\n",
       "      <td>77.662135</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>76.333450</td>\n",
       "      <td>139.987119</td>\n",
       "      <td>108.160285</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>46.563887</td>\n",
       "      <td>110.217556</td>\n",
       "      <td>78.390721</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>77.746811</td>\n",
       "      <td>141.400480</td>\n",
       "      <td>109.573645</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>10</th>\n",
       "      <td>127.279376</td>\n",
       "      <td>190.933045</td>\n",
       "      <td>159.106211</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>11</th>\n",
       "      <td>144.972381</td>\n",
       "      <td>208.626050</td>\n",
       "      <td>176.799216</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>12</th>\n",
       "      <td>141.800849</td>\n",
       "      <td>205.454518</td>\n",
       "      <td>173.627684</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>13</th>\n",
       "      <td>50.481506</td>\n",
       "      <td>114.135175</td>\n",
       "      <td>82.308340</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>14</th>\n",
       "      <td>86.298541</td>\n",
       "      <td>149.952210</td>\n",
       "      <td>118.125376</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>15</th>\n",
       "      <td>18.416184</td>\n",
       "      <td>82.069853</td>\n",
       "      <td>50.243019</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>16</th>\n",
       "      <td>26.135713</td>\n",
       "      <td>89.789382</td>\n",
       "      <td>57.962548</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>17</th>\n",
       "      <td>47.232894</td>\n",
       "      <td>110.886563</td>\n",
       "      <td>79.059728</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18</th>\n",
       "      <td>20.502737</td>\n",
       "      <td>84.156406</td>\n",
       "      <td>52.329571</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>19</th>\n",
       "      <td>38.033950</td>\n",
       "      <td>101.687619</td>\n",
       "      <td>69.860784</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>20</th>\n",
       "      <td>42.907392</td>\n",
       "      <td>106.561061</td>\n",
       "      <td>74.734227</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>21</th>\n",
       "      <td>72.138262</td>\n",
       "      <td>135.791931</td>\n",
       "      <td>103.965097</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>22</th>\n",
       "      <td>6.599422</td>\n",
       "      <td>70.253091</td>\n",
       "      <td>38.426257</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>23</th>\n",
       "      <td>30.271431</td>\n",
       "      <td>93.925100</td>\n",
       "      <td>62.098266</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24</th>\n",
       "      <td>20.206152</td>\n",
       "      <td>83.859820</td>\n",
       "      <td>52.032986</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25</th>\n",
       "      <td>24.580999</td>\n",
       "      <td>88.234668</td>\n",
       "      <td>56.407834</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>26</th>\n",
       "      <td>35.587216</td>\n",
       "      <td>99.240885</td>\n",
       "      <td>67.414050</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>27</th>\n",
       "      <td>56.611327</td>\n",
       "      <td>120.264996</td>\n",
       "      <td>88.438161</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>28</th>\n",
       "      <td>39.418052</td>\n",
       "      <td>103.071721</td>\n",
       "      <td>71.244886</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>29</th>\n",
       "      <td>42.058761</td>\n",
       "      <td>105.712430</td>\n",
       "      <td>73.885595</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30</th>\n",
       "      <td>51.714546</td>\n",
       "      <td>115.368215</td>\n",
       "      <td>83.541381</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>31</th>\n",
       "      <td>53.058314</td>\n",
       "      <td>116.711983</td>\n",
       "      <td>84.885149</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>32</th>\n",
       "      <td>71.872232</td>\n",
       "      <td>135.525901</td>\n",
       "      <td>103.699067</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>70.905868</td>\n",
       "      <td>134.559537</td>\n",
       "      <td>102.732703</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>34</th>\n",
       "      <td>83.963761</td>\n",
       "      <td>147.617429</td>\n",
       "      <td>115.790595</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>35</th>\n",
       "      <td>29.909616</td>\n",
       "      <td>93.563285</td>\n",
       "      <td>61.736451</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>36</th>\n",
       "      <td>74.361489</td>\n",
       "      <td>138.015158</td>\n",
       "      <td>106.188324</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>37</th>\n",
       "      <td>53.376763</td>\n",
       "      <td>117.030432</td>\n",
       "      <td>85.203598</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>38</th>\n",
       "      <td>87.457980</td>\n",
       "      <td>151.111649</td>\n",
       "      <td>119.284815</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>39</th>\n",
       "      <td>10.678280</td>\n",
       "      <td>74.331949</td>\n",
       "      <td>42.505114</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>40</th>\n",
       "      <td>42.696923</td>\n",
       "      <td>106.350592</td>\n",
       "      <td>74.523757</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>41</th>\n",
       "      <td>111.958065</td>\n",
       "      <td>175.611734</td>\n",
       "      <td>143.784900</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>42</th>\n",
       "      <td>-1.143311</td>\n",
       "      <td>62.510358</td>\n",
       "      <td>30.683523</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>43</th>\n",
       "      <td>5.349718</td>\n",
       "      <td>69.003387</td>\n",
       "      <td>37.176552</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>44</th>\n",
       "      <td>9.838799</td>\n",
       "      <td>73.492468</td>\n",
       "      <td>41.665633</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>45</th>\n",
       "      <td>52.605848</td>\n",
       "      <td>116.259517</td>\n",
       "      <td>84.432683</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>46</th>\n",
       "      <td>55.501775</td>\n",
       "      <td>119.155444</td>\n",
       "      <td>87.328609</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>47</th>\n",
       "      <td>83.392833</td>\n",
       "      <td>147.046502</td>\n",
       "      <td>115.219667</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>48</th>\n",
       "      <td>77.229590</td>\n",
       "      <td>140.883259</td>\n",
       "      <td>109.056425</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>49</th>\n",
       "      <td>91.411509</td>\n",
       "      <td>155.065178</td>\n",
       "      <td>123.238344</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50</th>\n",
       "      <td>136.491712</td>\n",
       "      <td>200.145381</td>\n",
       "      <td>168.318547</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>51</th>\n",
       "      <td>11.020909</td>\n",
       "      <td>74.674578</td>\n",
       "      <td>42.847744</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>52</th>\n",
       "      <td>19.823392</td>\n",
       "      <td>83.477061</td>\n",
       "      <td>51.650227</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>53</th>\n",
       "      <td>23.550759</td>\n",
       "      <td>87.204427</td>\n",
       "      <td>55.377593</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54</th>\n",
       "      <td>33.349199</td>\n",
       "      <td>97.002868</td>\n",
       "      <td>65.176033</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>55</th>\n",
       "      <td>28.177549</td>\n",
       "      <td>91.831218</td>\n",
       "      <td>60.004383</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>56</th>\n",
       "      <td>16.198516</td>\n",
       "      <td>79.852185</td>\n",
       "      <td>48.025350</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>57</th>\n",
       "      <td>26.633899</td>\n",
       "      <td>90.287568</td>\n",
       "      <td>58.460733</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>58</th>\n",
       "      <td>32.466076</td>\n",
       "      <td>96.119745</td>\n",
       "      <td>64.292910</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>59</th>\n",
       "      <td>25.017680</td>\n",
       "      <td>88.671349</td>\n",
       "      <td>56.844514</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>60</th>\n",
       "      <td>54.986690</td>\n",
       "      <td>118.640358</td>\n",
       "      <td>86.813524</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>61</th>\n",
       "      <td>10.891282</td>\n",
       "      <td>74.544951</td>\n",
       "      <td>42.718116</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>62</th>\n",
       "      <td>21.752546</td>\n",
       "      <td>85.406215</td>\n",
       "      <td>53.579381</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>63</th>\n",
       "      <td>39.562424</td>\n",
       "      <td>103.216093</td>\n",
       "      <td>71.389259</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>64</th>\n",
       "      <td>36.278476</td>\n",
       "      <td>99.932145</td>\n",
       "      <td>68.105310</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>65</th>\n",
       "      <td>57.797040</td>\n",
       "      <td>121.450709</td>\n",
       "      <td>89.623874</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>66</th>\n",
       "      <td>48.715722</td>\n",
       "      <td>112.369391</td>\n",
       "      <td>80.542556</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>67</th>\n",
       "      <td>166.197301</td>\n",
       "      <td>229.850970</td>\n",
       "      <td>198.024136</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>68</th>\n",
       "      <td>174.219022</td>\n",
       "      <td>237.872690</td>\n",
       "      <td>206.045856</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>69</th>\n",
       "      <td>40.036632</td>\n",
       "      <td>103.690301</td>\n",
       "      <td>71.863466</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>70</th>\n",
       "      <td>43.600973</td>\n",
       "      <td>107.254642</td>\n",
       "      <td>75.427807</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>71</th>\n",
       "      <td>36.470671</td>\n",
       "      <td>100.124340</td>\n",
       "      <td>68.297506</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>72</th>\n",
       "      <td>39.776007</td>\n",
       "      <td>103.429676</td>\n",
       "      <td>71.602841</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>73</th>\n",
       "      <td>69.832371</td>\n",
       "      <td>133.486040</td>\n",
       "      <td>101.659206</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>74</th>\n",
       "      <td>66.462950</td>\n",
       "      <td>130.116618</td>\n",
       "      <td>98.289784</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>75</th>\n",
       "      <td>17.487189</td>\n",
       "      <td>81.140858</td>\n",
       "      <td>49.314023</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>76</th>\n",
       "      <td>26.851487</td>\n",
       "      <td>90.505156</td>\n",
       "      <td>58.678322</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>77</th>\n",
       "      <td>33.617855</td>\n",
       "      <td>97.271524</td>\n",
       "      <td>65.444690</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>78</th>\n",
       "      <td>69.350999</td>\n",
       "      <td>133.004668</td>\n",
       "      <td>101.177833</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>79</th>\n",
       "      <td>39.123023</td>\n",
       "      <td>102.776692</td>\n",
       "      <td>70.949858</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>80</th>\n",
       "      <td>60.964842</td>\n",
       "      <td>124.618511</td>\n",
       "      <td>92.791676</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>81</th>\n",
       "      <td>68.738697</td>\n",
       "      <td>132.392366</td>\n",
       "      <td>100.565531</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>82</th>\n",
       "      <td>4.871861</td>\n",
       "      <td>68.525530</td>\n",
       "      <td>36.698696</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>83</th>\n",
       "      <td>9.207820</td>\n",
       "      <td>72.861489</td>\n",
       "      <td>41.034654</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>84</th>\n",
       "      <td>25.689770</td>\n",
       "      <td>89.343439</td>\n",
       "      <td>57.516605</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>85</th>\n",
       "      <td>97.310351</td>\n",
       "      <td>160.964019</td>\n",
       "      <td>129.137185</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>86</th>\n",
       "      <td>73.006692</td>\n",
       "      <td>136.660361</td>\n",
       "      <td>104.833527</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>87</th>\n",
       "      <td>24.991768</td>\n",
       "      <td>88.645437</td>\n",
       "      <td>56.818602</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>88</th>\n",
       "      <td>22.329133</td>\n",
       "      <td>85.982802</td>\n",
       "      <td>54.155968</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>89</th>\n",
       "      <td>22.187254</td>\n",
       "      <td>85.840923</td>\n",
       "      <td>54.014088</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>90</th>\n",
       "      <td>20.956491</td>\n",
       "      <td>84.610160</td>\n",
       "      <td>52.783325</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>91</th>\n",
       "      <td>23.169487</td>\n",
       "      <td>86.823156</td>\n",
       "      <td>54.996322</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>92</th>\n",
       "      <td>115.277935</td>\n",
       "      <td>178.931604</td>\n",
       "      <td>147.104769</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>93</th>\n",
       "      <td>119.243002</td>\n",
       "      <td>182.896671</td>\n",
       "      <td>151.069836</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>94</th>\n",
       "      <td>126.349293</td>\n",
       "      <td>190.002962</td>\n",
       "      <td>158.176127</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>95</th>\n",
       "      <td>139.978893</td>\n",
       "      <td>203.632562</td>\n",
       "      <td>171.805728</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>96</th>\n",
       "      <td>152.319844</td>\n",
       "      <td>215.973513</td>\n",
       "      <td>184.146679</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>97</th>\n",
       "      <td>153.597745</td>\n",
       "      <td>217.251414</td>\n",
       "      <td>185.424580</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>98</th>\n",
       "      <td>30.805944</td>\n",
       "      <td>94.459613</td>\n",
       "      <td>62.632779</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>99</th>\n",
       "      <td>48.623238</td>\n",
       "      <td>112.276907</td>\n",
       "      <td>80.450072</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>100</th>\n",
       "      <td>55.558103</td>\n",
       "      <td>119.211772</td>\n",
       "      <td>87.384937</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>101</th>\n",
       "      <td>57.001492</td>\n",
       "      <td>120.655161</td>\n",
       "      <td>88.828327</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>102</th>\n",
       "      <td>82.892219</td>\n",
       "      <td>146.545888</td>\n",
       "      <td>114.719053</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>103</th>\n",
       "      <td>86.696591</td>\n",
       "      <td>150.350260</td>\n",
       "      <td>118.523426</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>104</th>\n",
       "      <td>102.067307</td>\n",
       "      <td>165.720976</td>\n",
       "      <td>133.894141</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>105</th>\n",
       "      <td>95.962583</td>\n",
       "      <td>159.616252</td>\n",
       "      <td>127.789417</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>106</th>\n",
       "      <td>17.998419</td>\n",
       "      <td>81.652088</td>\n",
       "      <td>49.825254</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>107</th>\n",
       "      <td>21.316637</td>\n",
       "      <td>84.970306</td>\n",
       "      <td>53.143471</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>108</th>\n",
       "      <td>9.379292</td>\n",
       "      <td>73.032960</td>\n",
       "      <td>41.206126</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>109</th>\n",
       "      <td>17.401012</td>\n",
       "      <td>81.054681</td>\n",
       "      <td>49.227846</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>110</th>\n",
       "      <td>16.639297</td>\n",
       "      <td>80.292966</td>\n",
       "      <td>48.466131</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>111</th>\n",
       "      <td>15.110605</td>\n",
       "      <td>78.764274</td>\n",
       "      <td>46.937440</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>112</th>\n",
       "      <td>15.874951</td>\n",
       "      <td>79.528620</td>\n",
       "      <td>47.701785</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>113</th>\n",
       "      <td>34.948772</td>\n",
       "      <td>98.602441</td>\n",
       "      <td>66.775606</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>114</th>\n",
       "      <td>67.751904</td>\n",
       "      <td>131.405573</td>\n",
       "      <td>99.578739</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>115</th>\n",
       "      <td>67.938785</td>\n",
       "      <td>131.592454</td>\n",
       "      <td>99.765619</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>116</th>\n",
       "      <td>72.895782</td>\n",
       "      <td>136.549451</td>\n",
       "      <td>104.722616</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>117</th>\n",
       "      <td>100.959975</td>\n",
       "      <td>164.613644</td>\n",
       "      <td>132.786809</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>118</th>\n",
       "      <td>111.958774</td>\n",
       "      <td>175.612443</td>\n",
       "      <td>143.785608</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>119</th>\n",
       "      <td>36.809216</td>\n",
       "      <td>100.462885</td>\n",
       "      <td>68.636051</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>120</th>\n",
       "      <td>56.631122</td>\n",
       "      <td>120.284791</td>\n",
       "      <td>88.457956</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>121</th>\n",
       "      <td>46.499668</td>\n",
       "      <td>110.153337</td>\n",
       "      <td>78.326503</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>122</th>\n",
       "      <td>59.980722</td>\n",
       "      <td>123.634391</td>\n",
       "      <td>91.807556</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>123</th>\n",
       "      <td>41.432448</td>\n",
       "      <td>105.086117</td>\n",
       "      <td>73.259283</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>124</th>\n",
       "      <td>38.884931</td>\n",
       "      <td>102.538600</td>\n",
       "      <td>70.711765</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>125</th>\n",
       "      <td>43.335345</td>\n",
       "      <td>106.989014</td>\n",
       "      <td>75.162180</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>126</th>\n",
       "      <td>69.309567</td>\n",
       "      <td>132.963236</td>\n",
       "      <td>101.136402</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>127</th>\n",
       "      <td>93.964736</td>\n",
       "      <td>157.618405</td>\n",
       "      <td>125.791570</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>128</th>\n",
       "      <td>124.492397</td>\n",
       "      <td>188.146066</td>\n",
       "      <td>156.319232</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>129</th>\n",
       "      <td>39.606581</td>\n",
       "      <td>103.260250</td>\n",
       "      <td>71.433415</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>130</th>\n",
       "      <td>37.308046</td>\n",
       "      <td>100.961715</td>\n",
       "      <td>69.134880</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>131</th>\n",
       "      <td>135.791333</td>\n",
       "      <td>199.445002</td>\n",
       "      <td>167.618168</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>132</th>\n",
       "      <td>22.399146</td>\n",
       "      <td>86.052815</td>\n",
       "      <td>54.225981</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>133</th>\n",
       "      <td>24.524520</td>\n",
       "      <td>88.178188</td>\n",
       "      <td>56.351354</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>134</th>\n",
       "      <td>32.194758</td>\n",
       "      <td>95.848427</td>\n",
       "      <td>64.021593</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>135</th>\n",
       "      <td>63.202938</td>\n",
       "      <td>126.856607</td>\n",
       "      <td>95.029772</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>136</th>\n",
       "      <td>59.813773</td>\n",
       "      <td>123.467442</td>\n",
       "      <td>91.640607</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>137</th>\n",
       "      <td>71.396377</td>\n",
       "      <td>135.050046</td>\n",
       "      <td>103.223212</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>138</th>\n",
       "      <td>100.155115</td>\n",
       "      <td>163.808784</td>\n",
       "      <td>131.981950</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>139</th>\n",
       "      <td>95.709309</td>\n",
       "      <td>159.362978</td>\n",
       "      <td>127.536144</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>140</th>\n",
       "      <td>103.938926</td>\n",
       "      <td>167.592595</td>\n",
       "      <td>135.765760</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>141</th>\n",
       "      <td>23.843400</td>\n",
       "      <td>87.497069</td>\n",
       "      <td>55.670235</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>142</th>\n",
       "      <td>32.496168</td>\n",
       "      <td>96.149837</td>\n",
       "      <td>64.323002</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>143</th>\n",
       "      <td>35.526806</td>\n",
       "      <td>99.180475</td>\n",
       "      <td>67.353640</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>144</th>\n",
       "      <td>56.824584</td>\n",
       "      <td>120.478253</td>\n",
       "      <td>88.651418</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>145</th>\n",
       "      <td>82.576848</td>\n",
       "      <td>146.230517</td>\n",
       "      <td>114.403682</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>146</th>\n",
       "      <td>38.155788</td>\n",
       "      <td>101.809457</td>\n",
       "      <td>69.982622</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>147</th>\n",
       "      <td>40.807670</td>\n",
       "      <td>104.461339</td>\n",
       "      <td>72.634504</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>148</th>\n",
       "      <td>48.546372</td>\n",
       "      <td>112.200041</td>\n",
       "      <td>80.373206</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>149</th>\n",
       "      <td>29.248958</td>\n",
       "      <td>92.902627</td>\n",
       "      <td>61.075792</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>150</th>\n",
       "      <td>29.004805</td>\n",
       "      <td>92.658474</td>\n",
       "      <td>60.831639</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>151</th>\n",
       "      <td>18.430564</td>\n",
       "      <td>82.084233</td>\n",
       "      <td>50.257398</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>152</th>\n",
       "      <td>21.000832</td>\n",
       "      <td>84.654501</td>\n",
       "      <td>52.827666</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>153</th>\n",
       "      <td>46.194851</td>\n",
       "      <td>109.848520</td>\n",
       "      <td>78.021686</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>154</th>\n",
       "      <td>48.265968</td>\n",
       "      <td>111.919637</td>\n",
       "      <td>80.092803</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>155</th>\n",
       "      <td>9.391697</td>\n",
       "      <td>73.045366</td>\n",
       "      <td>41.218532</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>156</th>\n",
       "      <td>14.445763</td>\n",
       "      <td>78.099432</td>\n",
       "      <td>46.272597</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>157</th>\n",
       "      <td>30.633228</td>\n",
       "      <td>94.286897</td>\n",
       "      <td>62.460062</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>158</th>\n",
       "      <td>61.018747</td>\n",
       "      <td>124.672416</td>\n",
       "      <td>92.845581</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>159</th>\n",
       "      <td>66.751947</td>\n",
       "      <td>130.405616</td>\n",
       "      <td>98.578781</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>160</th>\n",
       "      <td>67.585657</td>\n",
       "      <td>131.239326</td>\n",
       "      <td>99.412492</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>161</th>\n",
       "      <td>14.340466</td>\n",
       "      <td>77.994135</td>\n",
       "      <td>46.167300</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>162</th>\n",
       "      <td>26.917952</td>\n",
       "      <td>90.571621</td>\n",
       "      <td>58.744786</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>163</th>\n",
       "      <td>12.602541</td>\n",
       "      <td>76.256210</td>\n",
       "      <td>44.429376</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>164</th>\n",
       "      <td>20.597874</td>\n",
       "      <td>84.251543</td>\n",
       "      <td>52.424708</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>165</th>\n",
       "      <td>45.854097</td>\n",
       "      <td>109.507766</td>\n",
       "      <td>77.680932</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>166</th>\n",
       "      <td>49.944379</td>\n",
       "      <td>113.598048</td>\n",
       "      <td>81.771213</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>167</th>\n",
       "      <td>155.983086</td>\n",
       "      <td>219.636755</td>\n",
       "      <td>187.809921</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>168</th>\n",
       "      <td>167.389053</td>\n",
       "      <td>231.042722</td>\n",
       "      <td>199.215887</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>169</th>\n",
       "      <td>27.027187</td>\n",
       "      <td>90.680856</td>\n",
       "      <td>58.854022</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>170</th>\n",
       "      <td>49.383075</td>\n",
       "      <td>113.036744</td>\n",
       "      <td>81.209910</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>171</th>\n",
       "      <td>38.154980</td>\n",
       "      <td>101.808649</td>\n",
       "      <td>69.981815</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>172</th>\n",
       "      <td>67.223377</td>\n",
       "      <td>130.877046</td>\n",
       "      <td>99.050212</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>173</th>\n",
       "      <td>31.947051</td>\n",
       "      <td>95.600720</td>\n",
       "      <td>63.773886</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>174</th>\n",
       "      <td>15.582756</td>\n",
       "      <td>79.236425</td>\n",
       "      <td>47.409590</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>175</th>\n",
       "      <td>12.025069</td>\n",
       "      <td>75.678738</td>\n",
       "      <td>43.851904</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>176</th>\n",
       "      <td>14.056810</td>\n",
       "      <td>77.710479</td>\n",
       "      <td>45.883645</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>177</th>\n",
       "      <td>12.658825</td>\n",
       "      <td>76.312493</td>\n",
       "      <td>44.485659</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>178</th>\n",
       "      <td>24.213599</td>\n",
       "      <td>87.867268</td>\n",
       "      <td>56.040433</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>179</th>\n",
       "      <td>32.721631</td>\n",
       "      <td>96.375300</td>\n",
       "      <td>64.548465</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>180</th>\n",
       "      <td>39.346996</td>\n",
       "      <td>103.000665</td>\n",
       "      <td>71.173830</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>181</th>\n",
       "      <td>38.192721</td>\n",
       "      <td>101.846390</td>\n",
       "      <td>70.019556</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>182</th>\n",
       "      <td>114.681768</td>\n",
       "      <td>178.335437</td>\n",
       "      <td>146.508602</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>183</th>\n",
       "      <td>149.044641</td>\n",
       "      <td>212.698310</td>\n",
       "      <td>180.871476</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>184</th>\n",
       "      <td>157.121770</td>\n",
       "      <td>220.775439</td>\n",
       "      <td>188.948604</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>185</th>\n",
       "      <td>10.857458</td>\n",
       "      <td>74.511127</td>\n",
       "      <td>42.684293</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>186</th>\n",
       "      <td>10.460553</td>\n",
       "      <td>74.114222</td>\n",
       "      <td>42.287388</td>\n",
       </tr>\n",
       "    <tr>\n",
       "      <th>187</th>\n",
       "      <td>12.252813</td>\n",
       "      <td>75.906482</td>\n",
       "      <td>44.079648</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>188</th>\n",
       "      <td>24.425080</td>\n",
       "      <td>88.078749</td>\n",
       "      <td>56.251915</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>189</th>\n",
       "      <td>20.535945</td>\n",
       "      <td>84.189614</td>\n",
       "      <td>52.362780</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>190</th>\n",
       "      <td>14.615185</td>\n",
       "      <td>78.268854</td>\n",
       "      <td>46.442019</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>191</th>\n",
       "      <td>-3.160238</td>\n",
       "      <td>60.493431</td>\n",
       "      <td>28.666596</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>192</th>\n",
       "      <td>-2.880603</td>\n",
       "      <td>60.773066</td>\n",
       "      <td>28.946231</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>193</th>\n",
       "      <td>63.541582</td>\n",
       "      <td>127.195251</td>\n",
       "      <td>95.368417</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>194</th>\n",
       "      <td>96.042162</td>\n",
       "      <td>159.695831</td>\n",
       "      <td>127.868996</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>195</th>\n",
       "      <td>117.100927</td>\n",
       "      <td>180.754596</td>\n",
       "      <td>148.927762</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>196</th>\n",
       "      <td>143.572260</td>\n",
       "      <td>207.225929</td>\n",
       "      <td>175.399094</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>197</th>\n",
       "      <td>61.358436</td>\n",
       "      <td>125.012105</td>\n",
       "      <td>93.185271</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>198</th>\n",
       "      <td>100.149367</td>\n",
       "      <td>163.803036</td>\n",
       "      <td>131.976201</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>199</th>\n",
       "      <td>68.947013</td>\n",
       "      <td>132.600682</td>\n",
       "      <td>100.773848</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "          lower       upper        mean\n",
       "0      9.910127   73.563796   41.736962\n",
       "1     25.881948   89.535617   57.708782\n",
       "2     42.761664  106.415333   74.588498\n",
       "3     53.725623  117.379292   85.552457\n",
       "4     34.964404   98.618072   66.791238\n",
       "5     52.772098  116.425766   84.598932\n",
       "6     45.835301  109.488970   77.662135\n",
       "7     76.333450  139.987119  108.160285\n",
       "8     46.563887  110.217556   78.390721\n",
       "9     77.746811  141.400480  109.573645\n",
       "10   127.279376  190.933045  159.106211\n",
       "11   144.972381  208.626050  176.799216\n",
       "12   141.800849  205.454518  173.627684\n",
       "13    50.481506  114.135175   82.308340\n",
       "14    86.298541  149.952210  118.125376\n",
       "15    18.416184   82.069853   50.243019\n",
       "16    26.135713   89.789382   57.962548\n",
       "17    47.232894  110.886563   79.059728\n",
       "18    20.502737   84.156406   52.329571\n",
       "19    38.033950  101.687619   69.860784\n",
       "20    42.907392  106.561061   74.734227\n",
       "21    72.138262  135.791931  103.965097\n",
       "22     6.599422   70.253091   38.426257\n",
       "23    30.271431   93.925100   62.098266\n",
       "24    20.206152   83.859820   52.032986\n",
       "25    24.580999   88.234668   56.407834\n",
       "26    35.587216   99.240885   67.414050\n",
       "27    56.611327  120.264996   88.438161\n",
       "28    39.418052  103.071721   71.244886\n",
       "29    42.058761  105.712430   73.885595\n",
       "30    51.714546  115.368215   83.541381\n",
       "31    53.058314  116.711983   84.885149\n",
       "32    71.872232  135.525901  103.699067\n",
       "33    70.905868  134.559537  102.732703\n",
       "34    83.963761  147.617429  115.790595\n",
       "35    29.909616   93.563285   61.736451\n",
       "36    74.361489  138.015158  106.188324\n",
       "37    53.376763  117.030432   85.203598\n",
       "38    87.457980  151.111649  119.284815\n",
       "39    10.678280   74.331949   42.505114\n",
       "40    42.696923  106.350592   74.523757\n",
       "41   111.958065  175.611734  143.784900\n",
       "42    -1.143311   62.510358   30.683523\n",
       "43     5.349718   69.003387   37.176552\n",
       "44     9.838799   73.492468   41.665633\n",
       "45    52.605848  116.259517   84.432683\n",
       "46    55.501775  119.155444   87.328609\
       "47    83.392833  147.046502  115.219667\n",
       "48    77.229590  140.883259  109.056425\n",
       "49    91.411509  155.065178  123.238344\n",
       "50   136.491712  200.145381  168.318547\n",
       "51    11.020909   74.674578   42.847744\n",
       "52    19.823392   83.477061   51.650227\n",
       "53    23.550759   87.204427   55.377593\n",
       "54    33.349199   97.002868   65.176033\n",
       "55    28.177549   91.831218   60.004383\n",
       "56    16.198516   79.852185   48.025350\n",
       "57    26.633899   90.287568   58.460733\n",
       "58    32.466076   96.119745   64.292910\n",
       "59    25.017680   88.671349   56.844514\n",
       "60    54.986690  118.640358   86.813524\n",
       "61    10.891282   74.544951   42.718116\n",
       "62    21.752546   85.406215   53.579381\n",
       "63    39.562424  103.216093   71.389259\n",
       "64    36.278476   99.932145   68.105310\n",
       "65    57.797040  121.450709   89.623874\n",
       "66    48.715722  112.369391   80.542556\n",
       "67   166.197301  229.850970  198.024136\n",
       "68   174.219022  237.872690  206.045856\n",
       "69    40.036632  103.690301   71.863466\n",
       "70    43.600973  107.254642   75.427807\n",
       "71    36.470671  100.124340   68.297506\n",
       "72    39.776007  103.429676   71.602841\n",
       "73    69.832371  133.486040  101.659206\n",
       "74    66.462950  130.116618   98.289784\n",
       "75    17.487189   81.140858   49.314023\n",
       "76    26.851487   90.505156   58.678322\n",
       "77    33.617855   97.271524   65.444690\n",
       "78    69.350999  133.004668  101.177833\n",
       "79    39.123023  102.776692   70.949858\n",
       "80    60.964842  124.618511   92.791676\n",
       "81    68.738697  132.392366  100.565531\n",
       "82     4.871861   68.525530   36.698696\n",
       "83     9.207820   72.861489   41.034654\n",
       "84    25.689770   89.343439   57.516605\n",
       "85    97.310351  160.964019  129.137185\n",
       "86    73.006692  136.660361  104.833527\n",
       "87    24.991768   88.645437   56.818602\n",
       "88    22.329133   85.982802   54.155968\n",
       "89    22.187254   85.840923   54.014088\n",
       "90    20.956491   84.610160   52.783325\n",
       "91    23.169487   86.823156   54.996322\n",
       "92   115.277935  178.931604  147.104769\n",
       "93   119.243002  182.896671  151.069836\n",
       "94   126.349293  190.002962  158.176127\n",
       "95   139.978893  203.632562  171.805728\n",
       "96   152.319844  215.973513  184.146679\n",
       "97   153.597745  217.251414  185.424580\n",
       "98    30.805944   94.459613   62.632779\n",
       "99    48.623238  112.276907   80.450072\n",
       "100   55.558103  119.211772   87.384937\n",
       "101   57.001492  120.655161   88.828327\n",
       "102   82.892219  146.545888  114.719053\n",
       "103   86.696591  150.350260  118.523426\n",
       "104  102.067307  165.720976  133.894141\n",
       "105   95.962583  159.616252  127.789417\n",
       "106   17.998419   81.652088   49.825254\n",
       "107   21.316637   84.970306   53.143471\n",
       "108    9.379292   73.032960   41.206126\n",
       "109   17.401012   81.054681   49.227846\n",
       "110   16.639297   80.292966   48.466131\n",
       "111   15.110605   78.764274   46.937440\n",
       "112   15.874951   79.528620   47.701785\n",
       "113   34.948772   98.602441   66.775606\n",
       "114   67.751904  131.405573   99.578739\n",
       "115   67.938785  131.592454   99.765619\n",
       "116   72.895782  136.549451  104.722616\n",
       "117  100.959975  164.613644  132.786809\n",
       "118  111.958774  175.612443  143.785608\n",
       "119   36.809216  100.462885   68.636051\n",
       "120   56.631122  120.284791   88.457956\n",
       "121   46.499668  110.153337   78.326503\n",
       "122   59.980722  123.634391   91.807556\n",
       "123   41.432448  105.086117   73.259283\n",
       "124   38.884931  102.538600   70.711765\n",
       "125   43.335345  106.989014   75.162180\n",
       "126   69.309567  132.963236  101.136402\n",
       "127   93.964736  157.618405  125.791570\n",
       "128  124.492397  188.146066  156.319232\n",
       "129   39.606581  103.260250   71.433415\n",
       "130   37.308046  100.961715   69.134880\n",
       "131  135.791333  199.445002  167.618168\n",
       "132   22.399146   86.052815   54.225981\n",
       "133   24.524520   88.178188   56.351354\n",
       "134   32.194758   95.848427   64.021593\n",
       "135   63.202938  126.856607   95.029772\n",
       "136   59.813773  123.467442   91.640607\n",
       "137   71.396377  135.050046  103.223212\n",
       "138  100.155115  163.808784  131.981950\n",
       "139   95.709309  159.362978  127.536144\n",
       "140  103.938926  167.592595  135.765760\n",
       "141   23.843400   87.497069   55.670235\n",
       "142   32.496168   96.149837   64.323002\n",
       "143   35.526806   99.180475   67.353640\n",
       "144   56.824584  120.478253   88.651418\n",
       "145   82.576848  146.230517  114.403682\n",
       "146   38.155788  101.809457   69.982622\n",
       "147   40.807670  104.461339   72.634504\n",
       "148   48.546372  112.200041   80.373206\n",
       "149   29.248958   92.902627   61.075792\n",
       "150   29.004805   92.658474   60.831639\n",
       "151   18.430564   82.084233   50.257398\n",
       "152   21.000832   84.654501   52.827666\n",
       "153   46.194851  109.848520   78.021686\n",
       "154   48.265968  111.919637   80.092803\n",
       "155    9.391697   73.045366   41.218532\n",
       "156   14.445763   78.099432   46.272597\n",
       "157   30.633228   94.286897   62.460062\n",
       "158   61.018747  124.672416   92.845581\n",
       "159   66.751947  130.405616   98.578781\n",
       "160   67.585657  131.239326   99.412492\n",
       "161   14.340466   77.994135   46.167300\n",
       "162   26.917952   90.571621   58.744786\n",
       "163   12.602541   76.256210   44.429376\n",
       "164   20.597874   84.251543   52.424708\n",
       "165   45.854097  109.507766   77.680932\n",
       "166   49.944379  113.598048   81.771213\n",
       "167  155.983086  219.636755  187.809921\n",
       "168  167.389053  231.042722  199.215887\n",
       "169   27.027187   90.680856   58.854022\n",
       "170   49.383075  113.036744   81.209910\n",
       "171   38.154980  101.808649   69.981815\n",
       "172   67.223377  130.877046   99.050212\n",
       "173   31.947051   95.600720   63.773886\n",
       "174   15.582756   79.236425   47.409590\n",
       "175   12.025069   75.678738   43.851904\n",
       "176   14.056810   77.710479   45.883645\n",
       "177   12.658825   76.312493   44.485659\n",
       "178   24.213599   87.867268   56.040433\n",
       "179   32.721631   96.375300   64.548465\n",
       "180   39.346996  103.000665   71.173830\n",
       "181   38.192721  101.846390   70.019556\n",
       "182  114.681768  178.335437  146.508602\n",
       "183  149.044641  212.698310  180.871476\n",
       "184  157.121770  220.775439  188.948604\n",
       "185   10.857458   74.511127   42.684293\n",
       "186   10.460553   74.114222   42.287388\n",
       "187   12.252813   75.906482   44.079648\n",
       "188   24.425080   88.078749   56.251915\n",
       "189   20.535945   84.189614   52.362780\n",
       "190   14.615185   78.268854   46.442019\n",
       "191   -3.160238   60.493431   28.666596\n",
       "192   -2.880603   60.773066   28.946231\n",
       "193   63.541582  127.195251   95.368417\n",
       "194   96.042162  159.695831  127.868996\n",
       "195  117.100927  180.754596  148.927762\n",
       "196  143.572260  207.225929  175.399094\n",
       "197   61.358436  125.012105   93.185271\n",
       "198  100.149367  163.803036  131.976201\n",
       "199   68.947013  132.600682  100.773848"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "pd.DataFrame(zip(lower_vet,upper_vet,output),columns=['lower','upper','mean'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3.8.5 ('base')",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  },
  "vscode": {
   "interpreter": {
    "hash": "7e8cbc31db87ef66faa56dc0153a3cc0cf513f996ecefcaa36993899447d62a6"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}